### Tutorial Link: https://pbpython.com/pandas_dtypes.html

### This notebook is a review of how to cleanup data using pandas

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("fin-data.csv")

In [4]:
df.head()

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002,Quest Industries,"$125,000.00","$162,500.00",30.00%,500,1,10,2015,Y
1,552278,Smith Plumbing,"$920,000.00","$1,012,000.00",10.00%,700,6,15,2014,Y
2,23477,ACME Industrial,"$50,000.00","$62,500.00",25.00%,125,3,29,2016,Y
3,24900,Brekke LTD,"$350,000.00","$490,000.00",4.00%,75,10,27,2015,Y
4,651029,Harbor Co,"$15,000.00","$12,750.00",-15.00%,Closed,2,2,2014,N


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Customer Number  5 non-null      int64 
 1   Customer Name    5 non-null      object
 2   2016             5 non-null      object
 3   2017             5 non-null      object
 4   Percent Growth   5 non-null      object
 5   Jan Units        5 non-null      object
 6   Month            5 non-null      int64 
 7   Day              5 non-null      int64 
 8   Year             5 non-null      int64 
 9   Active           5 non-null      object
dtypes: int64(4), object(6)
memory usage: 532.0+ bytes


#### Current issues: 
    1. Number is stored as a float
    2. 2016 and 2017 are stored as object/string ('$')
    3. Percent Growth also stored as object/string ('%', 'closed')
    4. Month, Day, Year should be stored as datetime objects instead of floats
    5. Active should be a Bool instead of an object/string

In [6]:
## 1. convert number from float to int
##  make sure to assign back to orig df since .astype() returns a copy
df['Customer Number'] = df['Customer Number'].astype('int')
df.dtypes

Customer Number     int32
Customer Name      object
2016               object
2017               object
Percent Growth     object
Jan Units          object
Month               int64
Day                 int64
Year                int64
Active             object
dtype: object

#### However astype() is not effective if the data can not easily be converted to a data type
    1. For example if astype is called on 2016 column, the '$' causes an Error
    2. a similar error occurs when trying to convert any of the other strings to datatypes like bool, int, or datatime
    3. Therefore we need to create custom datatype comversion functions


In [7]:
df["2016"].astype("float")

ValueError: could not convert string to float: '$125,000.00 '

### Defining Custom Type Conversion Functions

In [8]:
def convert_currency(val):
    """
    Convert a string number to a float
    """
    new_val = val.replace(',','').replace('$','')
    return float(new_val)

In [9]:
df["2016"].apply(convert_currency)

0    125000.0
1    920000.0
2     50000.0
3    350000.0
4     15000.0
Name: 2016, dtype: float64

##### Can also use a lambda function to acheive the same result with better performance:

In [10]:
df["2016"].apply(lambda val: val.replace(',','').replace('$','')).astype('float')

0    125000.0
1    920000.0
2     50000.0
3    350000.0
4     15000.0
Name: 2016, dtype: float64

##### Converting all the vals in 2016 and 2017 to floats:

In [11]:
df["2016"] = df["2016"].apply(convert_currency)
df["2017"] = df["2017"].apply(convert_currency)
df.dtypes

Customer Number      int32
Customer Name       object
2016               float64
2017               float64
Percent Growth      object
Jan Units           object
Month                int64
Day                  int64
Year                 int64
Active              object
dtype: object

In [12]:
def convert_percentage(val):
    new_val = val.replace('%','')
    return float(new_val) / 100

In [13]:
df["Percent Growth"].apply(lambda val: val.replace('%','')).astype('float') / 100

0    0.30
1    0.10
2    0.25
3    0.04
4   -0.15
Name: Percent Growth, dtype: float64

In [14]:
df["Percent Growth"] = df["Percent Growth"].apply(convert_percentage)

##### Converting 'Y' / 'N' to bool using np.where()

In [15]:
np.where(df["Active"] == 'Y', True, False)

array([ True,  True,  True,  True, False])

In [16]:
df["Active"] = np.where(df["Active"] == 'Y', True, False)

#### Pandas Helper Functions:
    1. The Jan Units column includes a "Closed" instance that will lead to errors if converted using astype()
    2. can use other pandas helper functions to clean up data in that column such as pd.to_numeric()
    3. pd.to_numeric() takes in a dataframe column, and has an arg errors which can be set to "coerce" to force any vals that cause errors to NaN

In [17]:
pd.to_numeric(df["Jan Units"], errors="coerce")

0    500.0
1    700.0
2    125.0
3     75.0
4      NaN
Name: Jan Units, dtype: float64

    4. we can then fill that NaN with .fillna(0)

In [18]:
pd.to_numeric(df["Jan Units"], errors='coerce').fillna(0)

0    500.0
1    700.0
2    125.0
3     75.0
4      0.0
Name: Jan Units, dtype: float64

    5. Finally assign the conversion to the column:

In [19]:
df["Jan Units"] = pd.to_numeric(df["Jan Units"], errors="coerce").fillna(0)

#### The last conversion we need to handle is the datetime conversion for the "Month", "Day", "Year" columns
    1. This can be accomplished by the pd.to_datetime() which can take in multiple columns and return a column with all the vals converted to a datetime type

In [20]:
pd.to_datetime(df[["Month", "Day", "Year"]])

0   2015-01-10
1   2014-06-15
2   2016-03-29
3   2015-10-27
4   2014-02-02
dtype: datetime64[ns]

    2. Set the output to a new column

In [21]:
df["Start Date"] = pd.to_datetime(df[["Month", "Day", "Year"]])

In [22]:
df.head()

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active,Start Date
0,10002,Quest Industries,125000.0,162500.0,0.30,500.0,1,10,2015,True,2015-01-10
1,552278,Smith Plumbing,920000.0,1012000.0,0.10,700.0,6,15,2014,True,2014-06-15
2,23477,ACME Industrial,50000.0,62500.0,0.25,125.0,3,29,2016,True,2016-03-29
3,24900,Brekke LTD,350000.0,490000.0,0.04,75.0,10,27,2015,True,2015-10-27
4,651029,Harbor Co,15000.0,12750.0,-0.15,0.0,2,2,2014,False,2014-02-02


### Apply datatype conversions all at once
    1. Now that we have defined all the different conversion functions we need to clean the dataset we can apply all the conversion functions (aside from the datetime in which we create a new column) at once
    

In [30]:
df_2 = pd.read_csv("fin-data.csv",
                   dtype={"Customer Number":"int64"},
                   converters={'2016': convert_currency,
                               '2017': convert_currency,
                               'Percent Growth': convert_percentage,
                               'Jan Units': lambda val: pd.to_numeric(val, errors='coerce'),
                               'Active': lambda val: np.where(val == 'Y', True, False)})

In [31]:
df_2.head()

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active
0,10002,Quest Industries,125000.0,162500.0,0.30,500.0,1,10,2015,True
1,552278,Smith Plumbing,920000.0,1012000.0,0.10,700.0,6,15,2014,True
2,23477,ACME Industrial,50000.0,62500.0,0.25,125.0,3,29,2016,True
3,24900,Brekke LTD,350000.0,490000.0,0.04,75.0,10,27,2015,True
4,651029,Harbor Co,15000.0,12750.0,-0.15,NaN,2,2,2014,False


In [32]:
df_2.dtypes

Customer Number      int64
Customer Name       object
2016               float64
2017               float64
Percent Growth     float64
Jan Units          float64
Month                int64
Day                  int64
Year                 int64
Active              object
dtype: object

In [33]:
df_2["Jan Units"] = df["Jan Units"].fillna(0)

In [34]:
df_2["Start Date"] = pd.to_datetime(df[["Month", "Day", "Year"]])

In [35]:
df_2.head()

,Customer Number,Customer Name,2016,2017,Percent Growth,Jan Units,Month,Day,Year,Active,Start Date
0,10002,Quest Industries,125000.0,162500.0,0.30,500.0,1,10,2015,True,2015-01-10
1,552278,Smith Plumbing,920000.0,1012000.0,0.10,700.0,6,15,2014,True,2014-06-15
2,23477,ACME Industrial,50000.0,62500.0,0.25,125.0,3,29,2016,True,2016-03-29
3,24900,Brekke LTD,350000.0,490000.0,0.04,75.0,10,27,2015,True,2015-10-27
4,651029,Harbor Co,15000.0,12750.0,-0.15,0.0,2,2,2014,False,2014-02-02


### Other tutorials: Cleaning up currency data
https://pbpython.com/currency-cleanup.html